In [1]:
import sys
import librosa
from math import *

/usr/local/lib/python2.7/dist-packages/librosa/__init__.py:35: FutureWarning: You are using librosa with Python 2. Please note that librosa 0.7 will be the last version to support Python 2, after which it will require Python 3 or later.
  FutureWarning)


In [ ]:
'''

In this file I have implemented Endpoint Detection using Energy of the signal.
Steps for endpoint detection
1. Cut the data into many segments.
2. Calculate energy for each of the segments and check is it is less than a threshold, the threshold which worked
    for most of the data was 0.001. Hamming window was used to obtain the energy of the signal in the window.
3. Find the speech start and the endpoints
4. Slice the audio file in that duration and obtain a cut version of the audio

In this, I used a window of size 25ms and half window size of 12.5ms


NOTE: This cut was applied to all the data points in the data and stored in a new folder and used thereafter for
      for the assignment. However, for the sake of demonstation, I have attached a sample wav file and obtained
      an endpointed version of the same.

'''

In [6]:
def readfile(fname):
    '''
    Function to read file from librosa package
    '''
    return librosa.load(fname, sr=None)


def cut_file(audiofile):
    '''
    Cutting the file into several chunks of the data
    '''
    global sr
    res = []
    for i in range(0, len(audiofile)-1, sr/80):
        res.append(audiofile[i:min(i+sr/40, len(audiofile)-1)])
    return res

In [7]:
# hamming window
def hamming(N, m):
    '''
    Hamming window formula
    '''
    return (0.54-0.46*cos(2*3.1415926*m/(N-1)))


def get_energy(window):
    '''
    Energy Calculation
    '''
    N = len(window)
    res = 0
    for m in range(0, N):
        res += window[m]*hamming(N, N-m)*window[m]*hamming(N, N-m)
    return res

In [8]:
filename = "sample"
audiofile, sr = readfile(filename+".wav")
windows = cut_file(audiofile)
sil_before = True
sil_start = 0
speech_start = 0

In [10]:
start_index = 0
end_index = 0

for i in range(len(windows)):
    if(get_energy(windows[i]) > 0.001):
        if(sil_before):
            sil_before = False
            speech_start = i
            start_index = round((i*12.5)*sr*1e-3)

    else:
        if(not sil_before):
            sil_before = True
            sil_start = i
            end_index = round((i*12.5)*sr*1e-3)

cut_data = audiofile[int(start_index):int(end_index)]
librosa.output.write_wav('sample_cut_data.wav', cut_data, sr, norm=False)

print("Number of data points in original sample: {}".format(len(audiofile)))
print("Number of data points in endpointed sample: {}".format(len(cut_data)))

Number of data points in original sample: 15019
Number of data points in endpointed sample: 6400


In [ ]:
'''
INFERENCE
We see that endpoint detection is really helpful in removing the extra part of the sound.
For example, this sample data wav file had 15019 samples and after endpoint detection the
number of sample points reduces to 6400 samples only.
'''